In [ ]:
import os
import random
import metpy
import datetime
import s3fs
import numpy as np
import pandas as pd
import xarray as xr
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
from pyresample import geometry, grid
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from skimage.filters import laplace
from skimage.filters import unsharp_mask
from skimage.transform import resize
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

fs = s3fs.S3FileSystem(anon = True)
model = hub.load("https://tfhub.dev/captain-pool/esrgan-tf2/1")
random.seed(42)

In [40]:
def control(img):
    """
    Returns a dictionary of errors for the given file without sharpening.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to evaluate the error of
    Precondition: file is a string
    """
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    control = resize(smallimg, (height, width))
    mae = mean_absolute_error(img, control)
    rmse = mean_squared_error(img, control, squared=False)
    return {'Control_MAE': mae, 'Control_RMSE': rmse}

def laplace_sharpening(img):
    """
    Returns a dictionary of errors for the given file using laplace sharpening.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to perform laplace sharpening on
    Precondition: file is a string
    """
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    blurryimg = resize(smallimg, (height, width))
    laplace_edges = laplace(blurryimg)
    sharpimg = blurryimg + laplace_edges
    mae = mean_absolute_error(img, sharpimg)
    rmse = mean_squared_error(img, sharpimg, squared=False)
    return {'LP_MAE': mae, 'LP_RMSE': rmse}

def unsharpmask_sharpening(img):
    """
    Returns a dictionary of errors for the given file using unsharpmask sharpening. To normalize images,
    each image is divided by its maximum value, and then multiplied by the same value after sharpening
    is completed.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to perform laplace sharpening on
    Precondition: file is a str
    """
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    blurryimg = resize(smallimg, (height, width))
    sharpimg = unsharp_mask(blurryimg/blurryimg.max(), radius=1, amount=1)*blurryimg.max()
    mae = mean_absolute_error(img, sharpimg)
    rmse = mean_squared_error(img, sharpimg, squared=False)
    return {'UM_MAE': mae, 'UM_RMSE': rmse}

def find_npy_file(path, file, band):
    """
    Returns the string name of another file from the same time for a given GOES band npy file,
    or None if there is no such file in the given directory.
    
    Parameter path: The path where the returned file is located in
    Precondition: path is a string to a directory relative to the current one, with .npy files
    
    Parameter file: A file from the same time the returned file should have
    Precondition: file is a string with the name of the original file, and includes the path
    
    Parameter band: GOES band the returned file should be from
    Precondition: band is a string of length 2
    """
    for x in os.listdir(path):
        if x[19:21] == band and file[23:37] == x[27:41]:
            return x

def update_dict(img, data):
    """
    Updates the dictionary with sharpening errors of a given file.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to perform laplace sharpening on
    Precondition: file is a string
    
    Parameter data: Dictionary containing sharpening errors
    Precondition: data is a dict
    """
    newdata = {}
    newdata.update(control(img))
    newdata.update(laplace_sharpening(img))
    newdata.update(unsharpmask_sharpening(img))
    newdata.update(esrgan_sharpening(img))
    data.update(newdata)

In [72]:
def control_img(img):
    """
    Returns a dictionary of errors for the given file without sharpening.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to evaluate the error of
    Precondition: file is a string
    """
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    control = resize(smallimg, (height, width))
    return control

def laplace_sharpening_img(img):
    """
    Returns a dictionary of errors for the given file using laplace sharpening.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to perform laplace sharpening on
    Precondition: file is a string
    """
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    blurryimg = resize(smallimg, (height, width))
    laplace_edges = laplace(blurryimg)
    sharpimg = blurryimg + laplace_edges
    return sharpimg

def unsharpmask_sharpening_img(img):
    """
    Returns a dictionary of errors for the given file using unsharpmask sharpening. To normalize images,
    each image is divided by its maximum value, and then multiplied by the same value after sharpening
    is completed.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to perform laplace sharpening on
    Precondition: file is a str
    """
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    blurryimg = resize(smallimg, (height, width))
    sharpimg = unsharp_mask(blurryimg/blurryimg.max(), radius=1, amount=1)*blurryimg.max()
    return sharpimg

def update_dict_img(img07, img14, data1, data2, data3, data4):
    """
    Updates the dictionary with sharpening errors of a given file.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to perform laplace sharpening on
    Precondition: file is a string
    
    Parameter data: Dictionary containing sharpening errors
    Precondition: data is a dict
    """
    data07 = control_img(img07)
    data14 = control_img(img14)
    data1.append(len(fire_threshold(data07, data14)))
    
    data07 = laplace_sharpening_img(img07)
    data14 = laplace_sharpening_img(img14)
    data2.append(len(fire_threshold(data07, data14)))
    
    data07 = unsharpmask_sharpening_img(img07)
    data14 = unsharpmask_sharpening_img(img14)
    data3.append(len(fire_threshold(data07, data14)))
    
#     data07 = esrgan_sharpening_img(img07)
#     data14 = esrgan_sharpening_img(img14)
#     data4.append(len(fire_threshold(data07, data14)))
    
def fire_threshold(img07, img14):
    m = (330-210)/(340-295)
    b = 210 - m*295
    points = []
    
    fire_mask = (img14) < (m*img07 + b)
    points = np.array(fire_mask.nonzero()).T
    
    for point in points:
        save = point[0]
        point[0] = point[1]
        point[1] = 500 - save
        
    return points

In [73]:
def load_image(array):
    """
    Returns loaded .npy file.
    
    Parameter path: Path to load .npy file from
    Precondition: path is a string
    """
    result = np.zeros((array.shape[0], array.shape[1], 3))
    result[:,:,0]= array
    result[:,:,1]= array
    result[:,:,2]= array
    return result

def preprocess_image(array):
    """
    Returns preprocessed input array.
    
    Parameter array: array to preprocess
    Precondition: array is a numpy array
    """
    hr_image = array
    hr_size = (tf.convert_to_tensor(hr_image.shape[:-1]) // 2) * 2
    hr_image = tf.image.crop_to_bounding_box(hr_image, 0, 0, hr_size[0], hr_size[1])
    hr_image = tf.cast(hr_image, tf.float32)
    return tf.expand_dims(hr_image, 0)

def downscale_image(image):
    """
    Returns low resolution image after scaling down input image using nearest neighbor downsampling.

    Parameter image: 3D of 4D tensor of preprocessed image
    Precondition: image is a tensor
    """
    height, width = image.shape
    result = np.zeros((image.shape[0]//2, image.shape[1]//2, 3))
    smallimg = resize(image, (round(height/2), round(width/2)))
    result[:,:,0] = smallimg
    result[:,:,1] = smallimg
    result[:,:,2] = smallimg
    lr_image = tf.expand_dims(result, 0)
    lr_image = tf.cast(lr_image, tf.float32)
    return lr_image

def esrgan_sharpening(image):
    """
    Returns dictionary containing errors after using the ESRGAN model.
    
    Parameter path: path of image not including file name
    Precondition: path is a string
    
    Parameter file: file of the image
    Precondition: file is a string of a .npy file
    """
    hr_image = preprocess_image(load_image(image))
    lr_image = downscale_image(image)
    fake_image = model(lr_image)
    fake_image = tf.squeeze(fake_image)

    hr_image = tf.squeeze(hr_image).numpy()
    lr_image = tf.squeeze(lr_image).numpy()
    lr_image = resize(lr_image, (hr_image.shape[0], hr_image.shape[1], 3)).ravel()
    fake_image = resize(fake_image.numpy(), (hr_image.shape[0], hr_image.shape[1], 3)).ravel()
    hr_image = hr_image.ravel()

    control_mae = mean_absolute_error(hr_image, lr_image)
    control_rmse = mean_squared_error(hr_image, lr_image, squared=False)

    mae = mean_absolute_error(hr_image, fake_image)
    rmse = mean_squared_error(hr_image, fake_image, squared=False)
    return {'Control_MAE*': control_mae, 'Control_RMSE*': control_rmse, 'SR_MAE': mae, 'SR_RMSE': rmse}

def esrgan_sharpening_img(image):
    """
    Returns dictionary containing errors after using the ESRGAN model.
    
    Parameter path: path of image not including file name
    Precondition: path is a string
    
    Parameter file: file of the image
    Precondition: file is a string of a .npy file
    """
    hr_image = preprocess_image(load_image(image))
    lr_image = downscale_image(image)
    fake_image = model(lr_image)
    fake_image = tf.squeeze(fake_image)

    hr_image = tf.squeeze(hr_image).numpy()
    lr_image = tf.squeeze(lr_image).numpy()
    lr_image = resize(lr_image, (hr_image.shape[0], hr_image.shape[1], 3)).ravel()
    fake_image = resize(fake_image.numpy(), (hr_image.shape[0], hr_image.shape[1], 3)).ravel()
    hr_image = hr_image.ravel()

    return hr_image

# Error

In [74]:
filedates = set()
files = []

for file in os.listdir('../../GOES_Files/npy_files'):
    filedates.add(file[27:41])

for file in os.listdir('../../GOES_Files/clear_sky_mask'):
    if file[23:37] in filedates:
        files.append(file)

In [ ]:
mean = []
std = []
path = '../../GOES_Files/npy_files/'
pathCM = '../../GOES_Files/clear_sky_mask/'
fires_initial = []
fires_final_control = []
fires_final_lp = []
fires_final_um = []
fires_final_ml = []

for n in range(5):
    dataCM = []
    x = 0

    while x < 10:
        try:
            CM = random.choice(files)
            files.remove(CM)

            file07 = find_npy_file('../../GOES_Files/npy_files', CM, '07')
            file14 = find_npy_file('../../GOES_Files/npy_files', CM, '14')

            img07 = np.load(path + file07)
            img14 = np.load(path + file14)

            mask = np.load(pathCM + CM)
            mask = mask.astype(bool)
            img07[mask]=0
            img14[mask]=0

            fires_initial.append(len(fire_threshold(img07, img14)))

            data07CM = {'File': file07, 'Band': file07[19:21]}
            update_dict(img07, data07CM)
            data14CM = {'File': file14, 'Band': file14[19:21]}
            update_dict(img14, data14CM)

            datadiffCM = {'Band': 'diff'}
            diffimgCM = img07 - img14
            update_dict(diffimgCM, datadiffCM)

            update_dict_img(img07, img14, fires_final_control, fires_final_lp, fires_final_um, fires_final_ml)

            dataCM.append(data07CM)
            dataCM.append(data14CM)
            dataCM.append(datadiffCM)
            
            x += 1
            
        except:
            print(CM)

    dfCM = pd.DataFrame(dataCM)
    mean.append(dfCM.groupby(dfCM['Band']).agg({'Control_MAE': 'mean', 'Control_RMSE': 'mean', 'Control_MAE*': 'mean', 'Control_RMSE*': 'mean',
                                'LP_MAE': 'mean', 'LP_RMSE': 'mean', 'UM_MAE': 'mean', 'UM_RMSE': 'mean', 'SR_MAE': 'mean', 'SR_RMSE': 'mean'}))
    std.append(dfCM.groupby(dfCM['Band']).agg({'Control_MAE': 'std', 'Control_RMSE': 'std', 'Control_MAE*': 'std', 'Control_RMSE*': 'std',
                            'LP_MAE': 'std', 'LP_RMSE': 'std', 'UM_MAE': 'std', 'UM_RMSE': 'std', 'SR_MAE': 'std', 'SR_RMSE': 'std'}))

OR_ABI-L2-ACMC-M3_G17_s20182962057189_e20182962059502_c20182962100071.npy
OR_ABI-L2-ACMC-M3_G17_s20182921747189_e20182921749562_c20182921750150.npy
OR_ABI-L2-ACMC-M3_G17_s20182900447189_e20182900449562_c20182900450134.npy


In [80]:
means = pd.DataFrame()
stds = pd.DataFrame()

for m in mean:
    means = pd.concat((m, means))
for s in std:
    stds = pd.concat((s, stds))

In [81]:
print('Average Mean Error')
means.groupby(means.index).agg({'Control_MAE': 'mean', 'Control_RMSE': 'mean', 'Control_MAE*': 'mean', 'Control_RMSE*': 'mean',
                                'LP_MAE': 'mean', 'LP_RMSE': 'mean', 'UM_MAE': 'mean', 'UM_RMSE': 'mean', 'SR_MAE': 'mean', 'SR_RMSE': 'mean'})

Average Mean Error


,Control_MAE,Control_RMSE,Control_MAE*,Control_RMSE*,LP_MAE,LP_RMSE,UM_MAE,UM_RMSE,SR_MAE,SR_RMSE
Band,,,,,,,,,,
07,17.866072,42.105091,17.866072,43.526791,15.397903,38.168564,15.511413,39.114735,12.515767,34.769905
14,17.733889,41.811852,17.733889,43.226864,15.284540,37.901878,15.391311,38.839695,12.431574,34.528862
diff,0.206071,0.450429,0.206071,0.484630,0.179445,0.407039,0.185575,0.418365,0.340513,0.565314


In [82]:
print("Average Standard Deviation of Error")
stds.groupby(stds.index).agg({'Control_MAE': 'mean', 'Control_RMSE': 'mean', 'Control_MAE*': 'mean', 'Control_RMSE*': 'mean',
                                'LP_MAE': 'mean', 'LP_RMSE': 'mean', 'UM_MAE': 'mean', 'UM_RMSE': 'mean', 'SR_MAE': 'mean', 'SR_RMSE': 'mean'})

Average Standard Deviation of Error


,Control_MAE,Control_RMSE,Control_MAE*,Control_RMSE*,LP_MAE,LP_RMSE,UM_MAE,UM_RMSE,SR_MAE,SR_RMSE
Band,,,,,,,,,,
07,6.495550,8.816499,6.495549,8.347182,5.538722,8.098306,5.686675,8.310637,4.487305,7.588385
14,6.469009,8.782341,6.469009,8.320427,5.515994,8.068251,5.660318,8.278485,4.467697,7.565650
diff,0.141847,0.310992,0.141847,0.336267,0.122162,0.282053,0.126477,0.289599,0.147932,0.314994


In [85]:
#mean number of fires in each image
print(np.mean(fires_initial))
print(np.mean(fires_final_control))
print(np.mean(fires_final_lp))
print(np.mean(fires_final_um))
# print(np.mean(fires_final_ml))

0.08333333333333333
0.0
392.3235294117647
0.0


In [89]:
fires_final_um

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]